<a href="https://colab.research.google.com/github/sahibpreetsingh12/100daysofcode/blob/master/dhoni_kohli_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -q keras
#!mkdir test

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cp -r /content/gdrive/My\ Drive/train/ /content/

In [68]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

training_data = []
IMG_SIZE = 150
def create_training_data():
   
    DATADIR = "train/"

    CATEGORIES = [ "dhoni","virat"]

    for category in CATEGORIES:  # do dhoni and virat

        path = os.path.join(DATADIR,category)  # create path to dhoni and virat
        
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dhoni 1=virat
        

        for img in os.listdir(path):  # iterate over each image per dhoni and virat
            img_array = cv2.imread(os.path.join(path,img))  # convert to array
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
            training_data.append([new_array, class_num])  # add this to our training_data
            

create_training_data()

print(len(training_data),type(training_data[1]))


73 <class 'list'>


In [69]:
import random

random.shuffle(training_data)
for sample in training_data[:10]:
    print(sample[1])

0
1
0
0
1
1
1
0
0
1


In [0]:
X = []
y = []

for features,label in training_data:
    X.append(features)
    #print(training_data[0])
    y.append(label)

#print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE,3)


In [71]:
print(X[0].shape)


(150, 150, 3)


In [78]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D



#X = X/255.0

model = Sequential()

model.add(Conv2D(128, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, epochs=15, validation_split=0.3)

Train on 51 samples, validate on 22 samples
Epoch 1/15
51/51 [==============================] - 1s 16ms/sample - loss: 3.8804 - acc: 0.4314 - val_loss: 1.2714 - val_acc: 0.5909
Epoch 2/15
51/51 [==============================] - 0s 5ms/sample - loss: 1.4195 - acc: 0.5294 - val_loss: 1.2359 - val_acc: 0.4091
Epoch 3/15
51/51 [==============================] - 0s 5ms/sample - loss: 0.8279 - acc: 0.5294 - val_loss: 0.6599 - val_acc: 0.6364
Epoch 4/15
51/51 [==============================] - 0s 5ms/sample - loss: 0.6763 - acc: 0.4902 - val_loss: 0.7047 - val_acc: 0.4091
Epoch 5/15
51/51 [==============================] - 0s 5ms/sample - loss: 0.6272 - acc: 0.6275 - val_loss: 0.6729 - val_acc: 0.6364
Epoch 6/15
51/51 [==============================] - 0s 5ms/sample - loss: 0.5743 - acc: 0.8824 - val_loss: 0.7265 - val_acc: 0.3182
Epoch 7/15
51/51 [==============================] - 0s 5ms/sample - loss: 0.5095 - acc: 0.6667 - val_loss: 0.6209 - val_acc: 0.6364
Epoch 8/15
51/51 [=============

In [74]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 148, 148, 128)     3584      
_________________________________________________________________
activation_29 (Activation)   (None, 148, 148, 128)     0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 49, 49, 128)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 47, 47, 256)       295168    
_________________________________________________________________
activation_30 (Activation)   (None, 47, 47, 256)       0         
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 15, 15, 256)       0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 57600)             0         
__________

In [0]:
model.save('cricketer.model')

In [0]:
import cv2
import tensorflow as tf

CATEGORIES = ["dhoni", "virat"]  # will use this to convert prediction num to string value


def prepare(filepath):
    IMG_SIZE = 150  # 150 in txt-based
    img_array = cv2.imread(filepath)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)  # return the image with shaping that TF wants.

In [0]:
model = tf.keras.models.load_model("cricketer.model")
prediction = model.predict([prepare('e3.jpeg')])

In [92]:
print(CATEGORIES[int(prediction[0][0])])

dhoni
